# One_sentiment_analysis_model

### Exploração e limpeza dos dados (EDA)

In [ ]:
# TODO
# dataset inicial? https://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis/data


### Transformação dos textos em números com TF-IDF

In [ ]:
# TODO

### Treinamento de modelo supervisionado (ex.: Logistic Regression, Naive Bayes)

In [ ]:
# TODO

### Métricas de desempenho (Acurácia, Precisão, Recall, F1-score)

In [ ]:
# TODO

### Serialização do modelo (joblib/pickle)

In [ ]:
# TODO